# Solutions for discussion January
## Single effect blocks for pymc3 and logistic
n_gene_in_block = 20, sample_size = 200000

In [1]:
import pandas as pd, numpy as np
import os
import pyreadr
from dsc.dsc_io import load_rds
cwd = os.path.expanduser("~/GIT/cnv-gene-mapping/data")
folder = "deletion_simu_30_shape0.777_scale0.843"
# name = "deletion.genes.block20.for_simu.sample.combined.genes.block1"

In [2]:
# +1 in gene index for gene name
fisher = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.sample.combined.genes.block1.fisher.gz", header = 0, sep = "\t")
fisher["p_deleterious"] = [k if i >= j else 1 for i,j,k in zip(fisher["d_c"], fisher["nd_c"], fisher["p"])]

In [3]:
fisher.shape

(2290, 7)

In [4]:
beta = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.block30.shape0.777.scale0.843.beta", header = None, names = ["beta"])

In [5]:
# index1 = pd.read_csv(f"{cwd}/deletion.block1.index.csv", header = None, sep = "\t", names = ["id1", "id2"])
index1 = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.sample.combined.block1.index.csv", header = None, sep = "\t", names = ["id1", "id2"])

In [6]:
index2 = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.sample.combined.block1.forsimu.index.csv", header = None, sep = "\t", names = ["is1", "is2"])

In [7]:
index2.tail()

,is1,is2
523,21904,21905
524,21936,21941
525,21979,21981
526,22088,22097
527,22337,22355


In [8]:
index = pd.concat([index1, index2], axis = 1)

In [9]:
index.head()

,id1,id2,is1,is2
0,0,7,0,7
1,8,17,14,23
2,18,22,93,97
3,23,36,164,177
4,37,44,229,236


In [10]:
index.shape

(528, 4)

In [11]:
index["number_effect"] = [np.count_nonzero(beta.loc[item[0]:item[1]]["beta"].tolist()) for idx, item in index2.iterrows()]

In [12]:
from collections import Counter
Counter(index["number_effect"])

Counter({1: 69, 0: 440, 5: 1, 2: 16, 3: 2})

In [13]:
idx_beta = index[index["number_effect"] >= 1][["is1", "is2", "id1", "id2"]]

In [29]:
idx_beta.head(10)

,is1,is2,id1,id2
0,0,7,0,7
3,164,177,23,36
8,853,855,51,53
19,1215,1273,81,139
20,1613,1617,140,144
22,1752,1770,146,164
27,2043,2043,174,174
36,2272,2294,186,208
38,2315,2327,218,230
54,2900,2929,266,295


In [15]:
idx_beta.shape

(88, 4)

In [16]:
beta_in_block = list()
fisher_in_block = list()
d_c = list()
d_nc = list()
nd_c = list()
nd_nc = list()
gene_in_block = list()
for idx, item in idx_beta.iterrows():
    beta_in_block.append(beta["beta"][item[0]:item[1]+1].tolist())
    fisher_in_block.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["p"].tolist())
    d_c.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["d_c"].tolist())
    d_nc.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["d_nc"].tolist())
    nd_c.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["nd_c"].tolist())
    nd_nc.append(fisher.set_index('gene').loc[[f"gene_{i+1}" for i in range(item[2], item[3]+1)]].reset_index()["nd_nc"].tolist())
    gene_in_block.append([f"gene_{i+1}" for i in range(item[2], item[3]+1)])

In [17]:
index1 = index[index["number_effect"] >= 1][["id1", "id2"]]
# index1 = index1.set_index([[i for i in range(index1.shape[0])]])

In [18]:
index1.tail()

,id1,id2
512,2133,2137
514,2140,2141
522,2188,2249
526,2261,2270
527,2271,2289


In [19]:
index1.shape

(88, 2)

In [20]:
logit_pymc3 = pd.DataFrame()
logit1 = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.sample.combined.genes.logit.all.blocks.pip.regional.csv", sep = "\t", header = None, names = ["logit1"])
pymc31 = pd.read_csv(f"{cwd}/{folder}/deletion.genes.block30.for_simu.sample.combined.genes.pymc3.chain_5.all.blocks.uniform.multi_seeds.pip.gz", sep = "\t", header = None, names = ["pymc3_pip1"])
i = 0
for idx, item in index1.iterrows():
    logit = pyreadr.read_r(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block30.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.logit.rds")
    pymc3 = pd.read_csv(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block30.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.pymc3.chain_1.uniform_new_bound.gz", usecols = [0], sep = "\t", header = 0)
    pymc3["block"] = f"block{idx}"
    susie = load_rds(f"{cwd}/{folder}/block_{item[0]}_{item[1]}/deletion.genes.block30.for_simu.sample.combined.genes.block_{item[0]}_{item[1]}.SuSiE.L_10.prior_0p005.susie.rds")["pip"]
    res = pd.concat([logit[None][["p1"]], pymc3[["inclusion_probability", "block"]]], axis = 1)
    res["logit1"] = logit1["logit1"].tolist()[item[0]:item[1]+1]
    res["pymc3_new"] = pymc31["pymc3_pip1"].tolist()[item[0]:item[1]+1]
    res["susie"] = susie
    res["beta"] = beta_in_block[i]
    res["fisher"] = fisher_in_block[i]
    res["gene"] = gene_in_block[i]
    res["d_c"] = d_c[i]
    res["d_nc"] = d_nc[i]
    res["nd_c"] = nd_c[i]
    res["nd_nc"] = nd_nc[i]
    res = res.rename(columns = {"p1": "logit", "inclusion_probability": "pymc3"})
    logit_pymc3 = pd.concat([logit_pymc3, res])
    i += 1

In [24]:
logit_pymc3["beta"] = logit_pymc3.apply(lambda x: 0 if x["beta"] == -0 else x["beta"], axis = 1)
logit_pymc3["beta"] = [np.round(x, 6) for x in logit_pymc3["beta"]]
logit_pymc3["logit"] = [np.round(x, 6) for x in logit_pymc3["logit"]]
logit_pymc3["logit1"] = [np.round(x, 6) for x in logit_pymc3["logit1"]]
logit_pymc3["pymc3"] = [np.round(x, 6) for x in logit_pymc3["pymc3"]]
logit_pymc3["pymc3_new"] = [np.round(x, 6) for x in logit_pymc3["pymc3_new"]]
logit_pymc3["susie"] = [np.round(x, 6) for x in logit_pymc3["susie"]]

In [25]:
logit_pymc3 = logit_pymc3[["block", "gene", "beta", "fisher", "d_c", "d_nc", "nd_c", "nd_nc", "logit", "logit1", "susie", "pymc3_new"]]

In [27]:
logit_pymc3.shape

(985, 12)

In [28]:
logit_pymc3.head()

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,logit1,susie,pymc3_new
0,block0,gene_1,0.000000,0.790466,6,11922,8,11920,0.125,0.015036,0.0,0.0174
1,block0,gene_2,0.000000,0.790466,6,11922,8,11920,0.125,0.015036,0.0,0.0139
2,block0,gene_3,0.000000,0.790466,6,11922,8,11920,0.125,0.015036,0.0,0.0174
3,block0,gene_4,0.000000,0.790466,6,11922,8,11920,0.125,0.015036,0.0,0.0168
4,block0,gene_5,0.552649,0.790466,6,11922,8,11920,0.125,0.015036,0.0,0.0190


In [31]:
logit_pymc3[logit_pymc3["block"] == "block8"]

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,logit1,susie,pymc3_new
0,block8,gene_52,1.957626,1.957547e-09,56,11872,9,11919,1.0,1.0,1.0,1.0000
1,block8,gene_53,0.000000,8.237290e-01,9,11919,11,11917,0.0,0.0,0.0,0.0148
2,block8,gene_54,0.000000,1.000000e+00,5,11923,4,11924,0.0,0.0,0.0,0.0251


In [32]:
block1 = list()
for i in sorted(set(logit_pymc3["block"]), key=logit_pymc3["block"].tolist().index):
    if min(logit_pymc3[logit_pymc3["block"] == i]["fisher"]) < 1e-3:# or np.count_nonzero(logit_pymc3[logit_pymc3["block"] == i]["beta"]) >= 2:
        block1.append(i)

In [33]:
logit_pymc3_1 = logit_pymc3[logit_pymc3["block"].isin(block1)]

In [34]:
logit_pymc3_1.head()

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,logit1,susie,pymc3_new
0,block8,gene_52,1.957626,1.957547e-09,56,11872,9,11919,1.000000,1.000000,1.000000,1.0000
1,block8,gene_53,0.000000,8.237290e-01,9,11919,11,11917,0.000000,0.000000,0.000000,0.0148
2,block8,gene_54,0.000000,1.000000e+00,5,11923,4,11924,0.000000,0.000000,0.000000,0.0251
0,block19,gene_82,0.000000,1.378009e-22,178,11750,39,11889,0.998205,0.998205,0.998559,0.0235
1,block19,gene_83,0.827460,1.286304e-07,31,11897,2,11926,0.000000,0.000000,0.014537,0.0481


In [36]:
logit_pymc3_1[logit_pymc3_1["block"] == "block8"]

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,logit1,susie,pymc3_new
0,block8,gene_52,1.957626,1.957547e-09,56,11872,9,11919,1.0,1.0,1.0,1.0000
1,block8,gene_53,0.000000,8.237290e-01,9,11919,11,11917,0.0,0.0,0.0,0.0148
2,block8,gene_54,0.000000,1.000000e+00,5,11923,4,11924,0.0,0.0,0.0,0.0251


In [37]:
logit_pymc3.to_csv(f"{cwd}/{folder}/PIP_comparison_0430.csv", sep = "\t", header = True, index = False)

In [39]:
sum(logit_pymc3[logit_pymc3["block"] == "block8"]["logit"])

1.0

In [40]:
logit_pymc3[logit_pymc3["fisher"] == min(logit_pymc3["fisher"])]

,block,gene,beta,fisher,d_c,d_nc,nd_c,nd_nc,logit,logit1,susie,pymc3_new
6,block450,gene_1822,0.000000,8.076649e-65,329,11599,31,11897,0.5,0.5,0.5,0.3719
7,block450,gene_1823,1.813771,8.076649e-65,329,11599,31,11897,0.5,0.5,0.5,0.6827
